In [ ]:
import pandas as pd

In [ ]:
hgmd = pd.read_csv("/data/projects/processBio/hgmd/variant_df.csv")

In [ ]:
hgmd

In [ ]:
# clinvar = pd.read_pickle("/data/projects/processBio/clinvar/clinvar/parseClinvarRepresentations.pkl")

In [ ]:
clinvarDF = pd.read_csv("/data/projects/processBio/clinvar/clinvar/parseClinvarWithVariant.csv")

In [ ]:
clinvarDF

In [ ]:
gnomad = pd.read_pickle("/data/projects/processBio/gnomad/gnomad.missenseVariants.pd.pkl")

In [ ]:
gnomad

In [ ]:
gnomadSymbols = gnomad[gnomad.seq != ""].INFO.apply(lambda i: i["SYMBOL"] if "SYMBOL" in i else "")

In [ ]:
gnomadSymbols

In [ ]:
clinvarSymbols = clinvarDF["GeneSymbol"]

In [ ]:
hgmdSymbols = hgmd["GENE"]

In [ ]:
symbols = pd.concat([clinvarSymbols,
                     hgmdSymbols,
                     gnomadSymbols])

In [ ]:
symbols.shape

In [ ]:
symbols.value_counts().shape, "{:.3f}".format(symbols.value_counts().mean())